nodes=[

    Node(id='34986', type='Person', properties=[Property(key='name', value='34986')]),
    
    Node(id='마라탕', type='Food')

    ]


rels=[
    
    Relationship(source=Node(id='34986'), target=Node(id='마라탕'), type='like')
    
    ]




In [1]:
!pip install langchain langchain_community langchain_openai openai neo4j python-dotenv --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━

In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Project_KG_Recommendation/

Mounted at /content/drive
/content/drive/MyDrive/Project_KG_Recommendation


In [23]:
from langchain.graphs import Neo4jGraph
from neo4j import GraphDatabase
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from openai import OpenAI

import pickle
import os
import numpy as np
import pandas as pd
import json

%load_ext autoreload
%autoreload 2
from KG_util import *
from KG_util import Word

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
graph = get_KG()

In [ ]:
with open('./KG.pkl', 'wb') as f:
        pickle.dump(graph, f)

AttributeError: Can't pickle local object 'Neo4jPool.open.<locals>.opener'

In [ ]:
graph_schema = graph.get_schema
print(graph_schema)
#graph.query()

Node properties:
Job {id: STRING, name: STRING, title: STRING, work_as: STRING, description: STRING, interest: STRING, location: STRING, position: STRING, companytype: STRING, duration: STRING, job: STRING, teaches: STRING}
Personality {id: STRING, name: STRING, trait: STRING}
Hobby {id: STRING, name: STRING, poston: STRING, activity: STRING, interest: STRING, genre: STRING, description: STRING, frequency: STRING, preference: STRING, like: STRING, analysis: STRING, annoyance: STRING, topic: STRING}
Node {id: STRING, name: STRING, conversation: STRING, value: STRING}
Person {id: STRING, name: STRING, age: STRING, gender: STRING, job: STRING, hobby: STRING, musicpreference: STRING, relationship: STRING, personality: STRING, profession: STRING, interest: STRING, favoritecolor: STRING, watchtv: STRING, major: STRING, mood: STRING, meticulous: STRING, likestvnews: STRING, dislikesfood: STRING, goal: STRING, occupation: STRING, favoritefood: STRING, dislike: STRING, volunteeractivity: STRING

In [6]:
target_users:list[str] = get_unique_target_people(graph, minimum_neighbors_for_target = 10)
target_users_adj_list:dict[str, list[dict[str,str]]] = get_adj_rel_to_node(target_users, graph)


unique_relationships = get_unique_relationships(graph)
prompt = [{'role':'system', 'content':'json 형식으로 출력해줘.'},{'role':'user', 'content':'주어진 동사가 얼마나 발화자의 관심과 기호를 반영하는지 0~1점 사이의 \
               점수를 부여해서 {단어:점수, 단어:점수, ...} 형태로 출력해줘. 예시는 다음과 같아. \
               [예시] \
                input: ["immersed in", "take", "visit", "interested in"], \
                output: {"immersed in":0.8, "take":0.5, "visit":0.6, "interested in":0.9}'
                + f'input으로 사용될 단어 리스트는 다음과 같아: {unique_relationships}'}]
rels_to_interest_score = get_json_from_prompt(prompt)

unique_nodes:list[str] = get_unique_nodes(graph)
#nodes_to_IDF:dict[str,float] = get_IDF_of_node(unique_nodes, graph)


#nodes_to_embeddings:dict[str, list[float]] = get_words_to_embeddings(unique_nodes)
#EMBEDDING_LEN = 1536


items = ['coke', 'travel_package', 'playstation', 'cookies', 'education service']
items_to_embeddings:dict = get_words_to_embeddings(items)
#EMBEDDING_LEN = 1536





100%|██████████| 5/5 [00:00<00:00,  5.07it/s]


In [ ]:
"""
with open('./nodes_to_embeddings.pkl', 'wb') as f:
        pickle.dump(nodes_to_embeddings, f)
with open('./nodes_to_IDF.pkl', 'wb') as f:
        pickle.dump(nodes_to_IDF, f)
"""

In [7]:
with open('./nodes_to_embeddings.pkl', 'rb') as f:
        nodes_to_embeddings:dict[str, list[float]] = pickle.load(f)
with open('./nodes_to_IDF.pkl', 'rb') as f:
        nodes_to_IDF:dict[str,float] = pickle.load(f)

In [17]:
test_idx = 7
print(f'활성유저 id:{target_users[test_idx]}')
get_adj_rel_to_node(target_users[test_idx], graph)


활성유저 id:43890


[{'HOBBY': 'Hobby1'},
 {'HOBBY': 'Coffee'},
 {'HOBBY': 'Musicalmovie'},
 {'HOBBY': 'Mobilegame'},
 {'HOBBY': 'Morningcoffee'},
 {'HOBBY': 'Children'},
 {'HOBBY': 'Uniquename'},
 {'HOBBY': 'Stargazing'},
 {'INTERESTED_IN': 'Hobby1'},
 {'INTERESTED_IN': 'Tired'},
 {'INTERESTED_IN': 'Movie'},
 {'INTERESTED_IN': 'Food'},
 {'INTERESTED_IN': '나'},
 {'INTERESTED_IN': 'Electronics'},
 {'INTERESTED_IN': 'Mobilegame'},
 {'INTERESTED_IN': 'Culture'},
 {'INTERESTED_IN': 'Autonomousdrivingcar'},
 {'INTERESTED_IN': 'Race'},
 {'INTERESTED_IN': 'Avatar2'},
 {'INTERESTED_IN': 'Car'},
 {'INTERESTED_IN': 'Christmasanimationmovie'},
 {'INTERESTED_IN': 'Christmasanimation'},
 {'INTERESTED_IN': '강아지 용품'},
 {'INTERESTED_IN': '살 생각'},
 {'INTERESTED_IN': 'Diaryforecast'},
 {'INTERESTED_IN': 'Firstlove'},
 {'KNOWS': '기상상태'},
 {'HAS': 'Cold'},
 {'PARTICIPATED_IN': '33'},
 {'WORK_AS': 'Work'},
 {'WORK_AS': 'Work_Overtime'},
 {'WORK_AS': 'Arbiter'},
 {'HOBY': 'Movieclassic'},
 {'VISIT': 'Location1'},
 {'VISIT': 'D

In [ ]:
candle_emb = get_words_to_embeddings('candle')
aroma_emb = get_words_to_embeddings('aromatherapy')
scent_emb = get_words_to_embeddings('scent')

print(cos_mapping(aroma_emb, scent_emb))
print(cos_mapping(aroma_emb, candle_emb))
print(cos_mapping(candle_emb, scent_emb))
#problem: 의미적으로 유사하다고 생각하는 것을 잘 못잡아낸다.
#영어로 하니까 잘되네 취소

100%|██████████| 1/1 [00:00<00:00,  4.72it/s]

0.4761545507302602
0.3649131521065857
0.4150509710867081


In [ ]:
cf_emb = get_words_to_embeddings('campfire')
candle_emb = get_words_to_embeddings('candle')
cos_mapping(cf_emb, candle_emb)

100%|██████████| 1/1 [00:00<00:00,  5.28it/s]


0.522847809030577

In [ ]:
cos_mapping(get_words_to_embeddings("Switch between handheld and console"), get_words_to_embeddings("Portability"))


100%|██████████| 1/1 [00:00<00:00,  7.38it/s]


0.3292779604794215

In [ ]:
test_user = target_users[test_idx]
print(test_user)
recommendation_items_score = calculate_simularity(target_users_adj_list[test_user], rels_to_interest_score, nodes_to_IDF, nodes_to_embeddings, items_to_embeddings)
recommendation_items_score

{'coke': 11.592217557331889,
 'travel_package': 11.568624559979908,
 'playstation': 11.048422982830663,
 'cookies': 11.80904000788163,
 'education service': 11.537511011364401}

In [15]:
item = 'candle'

item_KG = get_item_KG([item])
print(item_KG)
store_graph_to_db(graph, item_KG)
item_KG = KG_to_embeddings(item_KG)

{'candle': {'Aesthetic Appeal': ['Decorative purposes', 'Creating ambiance', 'Enhancing home decor'], 'Fragrance': ['Aromatherapy', 'Masking odors', 'Creating a cozy atmosphere'], 'Relaxation': ['Stress relief', 'Mood enhancement', 'Promoting relaxation'], 'Celebrations': ['Birthday parties', 'Dinner dates', 'Special occasions'], 'Lighting': ['Creating a warm glow', 'Enhancing the atmosphere', 'Providing light during power outages'], 'Gift Giving': ['Housewarming gifts', 'Hostess gifts', 'Thank you gifts'], 'Seasonal': ['Holiday decorations', 'Seasonal scents', 'Festive ambiance'], 'Cultural/Religious': ['Ceremonies', 'Prayers', 'Rituals']}}


100%|██████████| 1/1 [00:00<00:00,  4.47it/s]


In [19]:

test_user = target_users[test_idx]

relnode_test = get_relnode_embedding(get_adj_rel_to_node(test_user, graph))


100%|██████████| 1/1 [00:00<00:00,  4.91it/s]


In [ ]:
relnode_test

[HOBBY Hobby1,
 HOBBY Coffee,
 HOBBY Musicalmovie,
 HOBBY Mobilegame,
 HOBBY Morningcoffee,
 HOBBY Children,
 HOBBY Uniquename,
 HOBBY Stargazing,
 INTERESTED_IN Hobby1,
 INTERESTED_IN Tired,
 INTERESTED_IN Movie,
 INTERESTED_IN Food,
 INTERESTED_IN 나,
 INTERESTED_IN Electronics,
 INTERESTED_IN Mobilegame,
 INTERESTED_IN Culture,
 INTERESTED_IN Autonomousdrivingcar,
 INTERESTED_IN Race,
 INTERESTED_IN Avatar2,
 INTERESTED_IN Car,
 INTERESTED_IN Christmasanimationmovie,
 INTERESTED_IN Christmasanimation,
 INTERESTED_IN 강아지 용품,
 INTERESTED_IN 살 생각,
 INTERESTED_IN Diaryforecast,
 INTERESTED_IN Firstlove,
 KNOWS 기상상태,
 HAS Cold,
 PARTICIPATED_IN 33,
 WORK_AS Work,
 WORK_AS Work_Overtime,
 WORK_AS Arbiter,
 HOBY Movieclassic,
 VISIT Location1,
 VISIT Dinner,
 VISIT Busan,
 VISIT Jeju,
 VISIT Guam,
 VISIT Shanghai,
 LIKE Travel,
 LIKE Child1,
 LIKE Coffee,
 LIKE Fruit,
 LIKE 자동차,
 LIKE Electronics,
 LIKE Coffee,
 LIKE 대중교통,
 LIKE Meat,
 LIKE Child2,
 LIKE Meat,
 LIKE Summer,
 HATE Seafood,
 

In [24]:
score = score_of_items(relnode_test, item_KG, threshold = 0.08, store_on_db=True, user_id=test_user, graph=graph)




< Recommended Item >   ::   [Node Name]  ---  [Best Related Property]  ---  [Best Matched Instance]

< candle >   ::   HOBBY Coffee --- Gift Giving --- Hostess gifts  :  0.08413446438801479
< candle >   ::   HOBBY Stargazing --- Lighting --- Enhancing the atmosphere  :  0.08305373794668963
< candle >   ::   INTERESTED_IN Culture --- Cultural/Religious --- Rituals  :  0.14341150594691657
< candle >   ::   INTERESTED_IN Christmasanimationmovie --- Seasonal --- Holiday decorations  :  0.10589142437325695
< candle >   ::   INTERESTED_IN Christmasanimation --- Seasonal --- Holiday decorations  :  0.12545806666780385
< candle >   ::   VISIT Dinner --- Gift Giving --- Hostess gifts  :  0.11941695342113885
< candle >   ::   LIKE Electronics --- Lighting --- Enhancing the atmosphere  :  0.08325786662753501
< candle >   ::   LIKE Summer --- Seasonal --- Seasonal scents  :  0.20343781960937146
< candle >   ::   SEE Firstsnow --- Seasonal --- Seasonal scents  :  0.08372640484996159
< candle >   ::